In [1]:
!pip install pandas
!pip install xgboost
!pip install scikit-learn
!pip install sliceline

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost
import divisi

In [3]:
df_train = pd.read_csv("train-airline.csv")
df_test = pd.read_csv("test-airline.csv")
df_all = pd.concat([df_train, df_test])
# print(df_all.tail())
df_prepped = df_all.drop(columns=['index', 'id'])
# print(df_prepped.tail())

In [4]:
# Some more processing
def map_to_category(value):
    if value == 0:
        return "not applicable"
    elif value == 1:
        return "not satisfied"
    elif 2 <= value <= 4:
        return "neutral"
    elif value == 5:
        return "very satisfied"
    else:
        return "unknown" 
        
columns_5 = [
    "Inflight wifi service",
    "Departure/Arrival time convenient",
    "Ease of Online booking",
    "Gate location",
    "Food and drink",
    "Online boarding",
    "Seat comfort",
    "Inflight entertainment",
    "On-board service",
    "Leg room service",
    "Baggage handling",
    "Checkin service",
    "Inflight service",
    "Cleanliness"
]

df_prepped_cat = df_prepped.copy()

for column in columns_5:
    df_prepped_cat[column] = df_prepped_cat[column].apply(map_to_category)

# print(df_prepped_cat.head())

In [5]:
# summary stats for discretizing
# columns_sum1 = ["Age", "Flight Distance", "Departure Delay in Minutes", "Arrival Delay in Minutes"]
print(df_prepped_cat["Age"].describe())
print(df_prepped_cat["Flight Distance"].describe())

columns_sum = ["Departure Delay in Minutes", "Arrival Delay in Minutes"]
filtered_df = df_prepped_cat[(df_prepped_cat["Departure Delay in Minutes"] != 0) & (df_prepped_cat["Arrival Delay in Minutes"] != 0)]
summary_stats = filtered_df[columns_sum].describe()
print(summary_stats)

count    129880.000000
mean         39.427957
std          15.119360
min           7.000000
25%          27.000000
50%          40.000000
75%          51.000000
max          85.000000
Name: Age, dtype: float64
count    129880.000000
mean       1190.316392
std         997.452477
min          31.000000
25%         414.000000
50%         844.000000
75%        1744.000000
max        4983.000000
Name: Flight Distance, dtype: float64
       Departure Delay in Minutes  Arrival Delay in Minutes
count                43122.000000              42876.000000
mean                    42.503919                 42.417740
std                     56.499574                 57.349213
min                      1.000000                  1.000000
25%                     11.000000                 10.000000
50%                     24.000000                 23.000000
75%                     52.000000                 52.000000
max                   1592.000000               1584.000000


In [6]:
y = df_prepped['satisfaction'] == 'satisfied'

In [14]:
import xgboost as xgb
# Adapting the simple XGBoost model from example_adult to predict whether a passenger is satisfied
y = df_prepped_cat['satisfaction'] == 'satisfied'

X = df_prepped_cat.drop(columns=['satisfaction'])

X_continuous = X[['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']]
X_categorical = X.drop(columns=['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes'])

X_encoded = pd.get_dummies(X_categorical)
X = pd.concat([X_continuous, X_encoded], axis=1)

X = X.fillna(-999)  # Or use X.fillna(X.mean()) to fill with column means

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

# Initialize and train the XGBoost model
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

# Make predictions
model_preds = model.predict(X)
model_probs = model.predict_proba(X)
is_error = model_preds != y

print(f"Train + test error rate: {is_error.mean():.2%}")

Train + test error rate: 4.85%


In [17]:
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sliceline.slicefinder import Slicefinder
from optbinning import ContinuousOptimalBinning
from sklearn.preprocessing import StandardScaler

sample_fraction = 0.1  # Use 10% of the data
X_sample, _, is_error_sample, _ = train_test_split(X, is_error, test_size=(1 - sample_fraction), stratify=is_error)

scaler = StandardScaler()
X_sample_trans = pd.DataFrame(scaler.fit_transform(X_sample), columns=X_sample.columns)

columns_to_bin = ["Age", "Flight Distance", "Departure Delay in Minutes", "Arrival Delay in Minutes"]

for col in columns_to_bin:
    binning = ContinuousOptimalBinning()
    X_sample_trans[col] = binning.fit_transform(X_sample_trans[col], is_error_sample)

# Feature selection to retain the top 20 features using mutual information
selector = SelectKBest(mutual_info_classif, k=30)
X_sample_trans_selected = selector.fit_transform(X_sample_trans, is_error_sample)
selected_feature_names = X_sample_trans.columns[selector.get_support()]

X_sample_trans = SelectKBest(mutual_info_classif, k=30).fit_transform(X_sample_trans, is_error_sample)

sf = Slicefinder(
    alpha=0.95,
    k=1,
    max_l=3, 
    min_sup=50,  
    verbose=True
)

sf.fit(X_sample_trans, is_error_sample)

top_slices = sf.top_slices_

slice_results = []
for slice_ in top_slices:
    slice_dict = {feature: val for feature, val in zip(selected_feature_names, slice_)}
    slice_results.append(slice_dict)

(CVXPY) Jul 01 02:01:43 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.10.4067). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jul 01 02:01:43 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.10.4067). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


DEBUG:sliceline.slicefinder:Dropping 0/67 features below min_sup = 50.
DEBUG:sliceline.slicefinder:Initial top-K: count=2, max=0.399548, min=0.399548
DEBUG:sliceline.slicefinder:Level 2:
DEBUG:sliceline.slicefinder: -- generated paired slice candidates: 67 -> 2027
/Users/zxl/anaconda3/envs/slicing/lib/python3.9/site-packages/sliceline/slicefinder.py:365: RuntimeWarning: invalid value encountered in divide
  (slice_errors / slice_sizes) / self.average_error_ - 1
/Users/zxl/anaconda3/envs/slicing/lib/python3.9/site-packages/sliceline/slicefinder.py:366: RuntimeWarning: divide by zero encountered in divide
  ) - (1 - self.alpha) * (n_row_x_encoded / slice_sizes - 1)
DEBUG:sliceline.slicefinder: -- valid slices after eval: 1954/2027
DEBUG:sliceline.slicefinder: -- top-K: count=2, max=0.938280, min=0.938280
DEBUG:sliceline.slicefinder:Level 3:
DEBUG:sliceline.slicefinder: -- generated paired slice candidates: 1954 -> 32075
DEBUG:sliceline.slicefinder: -- valid slices after eval: 31150/32075

In [23]:
toshow = pd.DataFrame(slice_results, columns=selected_feature_names)
toshow[selected_feature_names[:10]]

,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Eco,Inflight wifi service_neutral,Inflight wifi service_not applicable,Inflight wifi service_not satisfied,Inflight wifi service_very satisfied,Departure/Arrival time convenient_neutral
0,None,None,None,NaN,2.087603,None,None,1.042157,None,None
1,None,None,None,-2.087603,NaN,None,None,1.042157,None,None


In [24]:
toshow[selected_feature_names[10:20]]

,Departure/Arrival time convenient_not satisfied,Ease of Online booking_neutral,Ease of Online booking_not applicable,Ease of Online booking_not satisfied,Ease of Online booking_very satisfied,Gate location_not applicable,Gate location_not satisfied,Food and drink_not satisfied,Online boarding_neutral,Online boarding_not satisfied
0,None,None,None,None,None,None,None,None,None,0.697898
1,None,None,None,None,None,None,None,None,None,0.697898


In [29]:
toshow[selected_feature_names[20:30]]

,Online boarding_very satisfied,Seat comfort_very satisfied,Inflight entertainment_neutral,Inflight entertainment_very satisfied,On-board service_very satisfied,Baggage handling_neutral,Baggage handling_not satisfied,Baggage handling_very satisfied,Inflight service_neutral,Cleanliness_very satisfied
0,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [05:32<00:00,  3.01it/s]


In [15]:
# Discretize the dataset using a different method per-column so that we can perform slicing.

discrete_df = divisi.discretization.discretize_data(df_prepped_cat, {
    'Gender': { "method": "unique" }, 
    'Customer Type': { "method": "unique" }, 
    'Age': { "method": "bin", "bins": [25, 40, 50, 85] }, 
    'Type of Travel': { "method": "unique" }, 
    'Class': { "method": "unique" }, 
    'Flight Distance': { "method": "bin", "bins": [500, 1000, 2000, 130000] }, #change this
    'Inflight wifi service': {  "method": "unique" }, 
    'Departure/Arrival time convenient': {  "method": "unique" }, 
    # break
    'Ease of Online booking': {  "method": "unique" }, 
    'Gate location': {  "method": "unique" }, 
    'Food and drink': {  "method": "unique" }, 
    'Online boarding': { "method": "unique" }, 
    'Seat comfort': { "method": "unique" },
    # line break, too much
    'Inflight entertainment': { "method": "unique" }, 
    'On-board service': { "method": "unique" }, 
    'Leg room service': { "method": "unique" }, 
    'Baggage handling': { "method": "unique" }, 
    'Checkin service': { "method": "unique" }, 
    # another break
    'Inflight service': { "method": "unique" }, 
    'Cleanliness': { "method": "unique" }, 
    # others
    'Departure Delay in Minutes': { "method": "bin", "bins": [0, 15, 30, 60, 1000] }, 
    'Arrival Delay in Minutes': { "method": "bin", "bins": [0, 15, 30, 60, 1000] }, 
})

In [ ]:
# Create a widget to control the slice finder and to show metrics for each slice
w = divisi.widget.SliceFinderWidget(discrete_df, metrics={
    "satisfaction": y.values,
    "Model Prob.": model_probs[:,1],
    "Error": is_error.values,
})
w